In [1]:
!pip install duckdb rdkit mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 84.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 53.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.3 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176721 sha256=b88eea94af0ab9f60cbec1acb333bc76bfb36a321015bc687bb38c2bc47d0fb1
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [2]:
import pandas as pd
import joblib
import duckdb

from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Crippen, rdMolDescriptors
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
import numpy as np
from mordred import Calculator, descriptors

import multiprocessing
import concurrent.futures


In [3]:
# train_path = '/kaggle/input/leash-BELKA/train.parquet'
# # test_path = '/kaggle/input/leash-BELKA/test.parquet'
# range = 8000000
# con = duckdb.connect()

# df = con.query(f"""(SELECT *
#                         FROM parquet_scan('{train_path}')
#                         WHERE binds = 0
#                         ORDER BY random()
#                         LIMIT {range})
#                         UNION ALL
#                         (SELECT *
#                         FROM parquet_scan('{train_path}')
#                         WHERE binds = 1
#                         ORDER BY random()
#                         LIMIT {range})""").df()

# con.close()
# df.to_csv('somepart.csv')

In [4]:
df = pd.read_csv('/kaggle/input/leash-bio-belka-chunk/somepart_normalized/somepart_normalized.csv')

In [5]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Concatenate, Embedding, GlobalAveragePooling1D, Conv1D, GlobalMaxPooling1D, Attention
# from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import sklearn.metrics
# from sklearn.preprocessing import RobustScaler, MinMaxScaler, MaxAbsScaler
import json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight


2024-07-03 18:18:51.656956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 18:18:51.657062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 18:18:51.785307: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
smile_vector = tf.keras.layers.TextVectorization(max_tokens=None, split='character', standardize=None, output_sequence_length=70)
smile_vector.set_vocabulary(["", "[UNK]", "c", "C", "1", ")", "(", "O", "2", "N", "=", "n", "-", "l", "]", "[", "@", "H", "F", ".", "3", "s", "B", "r", "S", "#", "+", "o", "I", "4", "/", "5", "i"])

In [7]:
# df['binds'].values.flatten()

In [19]:
target_protein_name = 'sEH'
data = df[df['protein_name'] == target_protein_name].sample(frac=1)
len(data)

1722862

In [20]:
buildingblock1_smiles_vec = smile_vector(data['buildingblock1_smiles'].values)
buildingblock2_smiles_vec = smile_vector(data['buildingblock2_smiles'].values)
buildingblock3_smiles_vec = smile_vector(data['buildingblock3_smiles'].values)

In [21]:
p1 = int(6*len(data)/10)
p2 = int(9*len(data)/10)
print(p1, p2, len(data))

1033717 1550575 1722862


In [22]:
# Balance Data w/ Weights
class_weights = compute_class_weight('balanced', classes=np.unique(data['binds'].values.flatten()), y=data['binds'].values.flatten())
class_weight_dict = dict(enumerate(class_weights))

In [23]:
vector_length = buildingblock3_smiles_vec.shape[1]
vector_length

70

In [24]:
data_cols = [
 'b1LogP',
 'b1MolMR',
 'b1TPSA',
 'b1FractionCSP3',
 'b1NumHeteroatoms',
 'b1MolecularWeight',
 'b1ExactMass',
 'b1NumRotatableBonds',
 'b1NumValenceElectrons',
 'b1BondCount',
 'b1NumHAcceptors',

 'b2LogP',
 'b2MolMR',
 'b2TPSA',
 'b2FractionCSP3',
 'b2NumValenceElectrons',
 'b2MolecularWeight',
 'b2ExactMass',
 'b2NumRotatableBonds',
 'b2BondCount',
 'b2NumHeteroatoms',
 'b2NumHAcceptors',
 
 'b3LogP',
 'b3MolMR',
 'b3TPSA',
 'b3FractionCSP3',
 'b3NumHAcceptors',
 'b3MolecularWeight',
 'b3ExactMass',
 'b3BondCount',
 'b3NumValenceElectrons',
 'b3NumRotatableBonds',
 'b3NumHeteroatoms',
 'b3NumAromaticBonds',
]

In [ ]:
# Define inputs
input1 = Input(shape=(vector_length,))
input2 = Input(shape=(vector_length,))
input3 = Input(shape=(vector_length,))
input4 = Input(shape=(len(data_cols),))

# Embedding layer
embedding_dim = 128  # Increase embedding dimension
embedded1 = Embedding(input_dim=len(smile_vector.get_vocabulary()), output_dim=embedding_dim)(input1)
embedded2 = Embedding(input_dim=len(smile_vector.get_vocabulary()), output_dim=embedding_dim)(input2)
embedded3 = Embedding(input_dim=len(smile_vector.get_vocabulary()), output_dim=embedding_dim)(input3)

# Convolutional and pooling layers
conv_filters = 128
conv1 = Conv1D(filters=conv_filters, kernel_size=5, activation='relu', padding='same')(embedded1)
conv2 = Conv1D(filters=conv_filters, kernel_size=5, activation='relu', padding='same')(embedded2)
conv3 = Conv1D(filters=conv_filters, kernel_size=5, activation='relu', padding='same')(embedded3)

# Attention layers
attention1 = Attention()([conv1, conv1])
attention2 = Attention()([conv2, conv2])
attention3 = Attention()([conv3, conv3])

# Pooling layers
pooled1 = GlobalMaxPooling1D()(attention1)
pooled2 = GlobalMaxPooling1D()(attention2)
pooled3 = GlobalMaxPooling1D()(attention3)

# Concatenate inputs
concat = Concatenate()([pooled1, pooled2, pooled3])
# concat = Concatenate()([embedded1, embedded2, embedded3])

# Dense layers
dense1 = Dense(256, activation='relu')(concat)
dropout1 = Dropout(0.3)(dense1)

# Concatenate with numerical input
concat_with_numerical = Concatenate()([dropout1, input4])

dense2 = Dense(128, activation='relu')(concat_with_numerical)
dropout2 = Dropout(0.1)(dense2)
batch_norm = BatchNormalization()(dropout2)
dense3 = Dense(64, activation='relu')(batch_norm)
output = Dense(1, activation='sigmoid')(dense3)  # For binary classification

# Create the model
model = Model(inputs=[input1, input2, input3, input4], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

# Train the model with validation data
history = model.fit(
  [buildingblock1_smiles_vec[:p1], buildingblock2_smiles_vec[:p1], buildingblock3_smiles_vec[:p1], data[data_cols][:p1].values],
  data['binds'][:p1].values.flatten(),
  validation_data=(
      [buildingblock1_smiles_vec[p1:p2], buildingblock2_smiles_vec[p1:p2], buildingblock3_smiles_vec[p1:p2], data[data_cols][p1:p2].values],
      data['binds'][p1:p2].values.flatten()),
  batch_size=30,  # Decrease batch size for more updates
  epochs=100,
  class_weight=class_weight_dict,
  callbacks=[early_stopping, reduce_lr])

Epoch 1/100
34458/34458 ━━━━━━━━━━━━━━━━━━━━ 203s 6ms/step - accuracy: 0.9287 - loss: 0.1879 - val_accuracy: 0.9650 - val_loss: 0.1009 - learning_rate: 0.0010
Epoch 2/100
34458/34458 ━━━━━━━━━━━━━━━━━━━━ 188s 5ms/step - accuracy: 0.9626 - loss: 0.1073 - val_accuracy: 0.9652 - val_loss: 0.1021 - learning_rate: 0.0010
Epoch 3/100
34458/34458 ━━━━━━━━━━━━━━━━━━━━ 187s 5ms/step - accuracy: 0.9664 - loss: 0.0977 - val_accuracy: 0.9711 - val_loss: 0.0859 - learning_rate: 0.0010
Epoch 4/100
34458/34458 ━━━━━━━━━━━━━━━━━━━━ 190s 6ms/step - accuracy: 0.9684 - loss: 0.0924 - val_accuracy: 0.9706 - val_loss: 0.0847 - learning_rate: 0.0010
Epoch 5/100
34458/34458 ━━━━━━━━━━━━━━━━━━━━ 188s 5ms/step - accuracy: 0.9694 - loss: 0.0886 - val_accuracy: 0.9727 - val_loss: 0.0809 - learning_rate: 0.0010
Epoch 6/100
34458/34458 ━━━━━━━━━━━━━━━━━━━━ 190s 6ms/step - accuracy: 0.9708 - loss: 0.0850 - val_accuracy: 0.9725 - val_loss: 0.0801 - learning_rate: 0.0010
Epoch 7/100
34458/34458 ━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate([buildingblock1_smiles_vec[p2:], buildingblock2_smiles_vec[p2:], buildingblock3_smiles_vec[p2:], data[data_cols][p2:]], data['binds'][p2:].values.flatten())
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test Loss: {loss:.4f}')


In [ ]:
model.save('/kaggle/working/'+target_protein_name+'.keras')